# USB Protocol Analysis

Use the dropdown below to select a specific capture file (`source_file`) to analyze. Select 'All' to view aggregated data from all captures.


In [1]:
import polars as pl
import ipywidgets as widgets
from ipywidgets import VBox, Dropdown, Output, HTML
from IPython.display import display
import sys
from pathlib import Path
import warnings
import matplotlib.pyplot as plt
import numpy as np

warnings.filterwarnings('ignore', category=UserWarning, module='polars')


# --- Path Setup ---
# To make imports robust, we'll find the project root and add the scripts directory to the path.
# The project root is expected to contain the 'pyproject.toml' file.
try:
    # Start from the current working directory and go up until we find the root
    project_root = Path.cwd()
    while not (project_root / 'pyproject.toml').exists():
        if project_root == project_root.parent:
            raise FileNotFoundError("Reached filesystem root, 'pyproject.toml' not found.")
        project_root = project_root.parent
    
    analysis_scripts_path = project_root / 'analysis' / 'scripts'
    if str(analysis_scripts_path) not in sys.path:
        sys.path.insert(0, str(analysis_scripts_path))
    
    print(f"✅ Project root found at: {project_root}")
    print(f"✅ Added to sys.path: {analysis_scripts_path}")

except FileNotFoundError:
    print("❌ Could not determine project root. Imports may fail.")
    print(f"   Current working directory: {Path.cwd()}")


# --- Import and Data Load ---
try:
    import helpers
    print("✅ Successfully imported helpers.py")
except ImportError as e:
    print("❌ Could not import helpers.py. This might be due to a missing dependency in the script itself.")
    print(f"   Detailed error: {e}")
    helpers = None

# Load data
df = None
if helpers:
    try:
        # The master dataset is at the project root.
        df = helpers.load_master_dataset(project_root / 'usb_master_dataset.parquet')
    except FileNotFoundError as e:
        print(f"❌ {e}")


def plot_adc_data(filtered_df, selected_file):
    """Extract and plot ADC data from the filtered dataset."""
    # Get only payload data and parse it
    payload_df = filtered_df.filter(pl.col('payload_hex') != '')
    if len(payload_df) == 0:
        print("No payload data found for ADC plotting.")
        return
    
    # Add parsed packet data to get ADC values
    parsed_df = helpers.add_parsed_packet_data(payload_df)
    
    # Filter for only ADC data packets
    adc_df = parsed_df.filter(pl.col('packet_type') == 'SimpleAdcData')
    
    if len(adc_df) == 0:
        print("No ADC data found in this selection.")
        return
    
    # Extract data for plotting
    timestamps = adc_df['timestamp'].to_list()
    vbus_v = adc_df['vbus_v'].to_list()
    ibus_a_raw = adc_df['ibus_a'].to_list()
    ibus_a = [abs(val) for val in ibus_a_raw]  # Use absolute value for current
    power_w_raw = adc_df['power_w'].to_list()
    power_w = [abs(val) for val in power_w_raw]  # Use absolute value for power too
    temp_c = adc_df['temp_c'].to_list()
    
    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'KM003C ADC Measurements - {selected_file}', fontsize=16, fontweight='bold')
    
    # Plot voltage
    axes[0,0].plot(timestamps, vbus_v, 'b-', linewidth=1.5, alpha=0.8)
    axes[0,0].set_title('VBUS Voltage', fontsize=12, fontweight='bold')
    axes[0,0].set_ylabel('Voltage (V)')
    axes[0,0].grid(True, alpha=0.3)
    axes[0,0].set_ylim(bottom=0)
    
    # Plot current (absolute value)
    axes[0,1].plot(timestamps, ibus_a, 'r-', linewidth=1.5, alpha=0.8)
    axes[0,1].set_title('IBUS Current (Absolute)', fontsize=12, fontweight='bold')
    axes[0,1].set_ylabel('Current (A)')
    axes[0,1].grid(True, alpha=0.3)
    axes[0,1].set_ylim(bottom=0)
    
    # Plot power (absolute value)
    axes[1,0].plot(timestamps, power_w, 'g-', linewidth=1.5, alpha=0.8)
    axes[1,0].set_title('Power (Absolute)', fontsize=12, fontweight='bold')
    axes[1,0].set_ylabel('Power (W)')
    axes[1,0].set_xlabel('Time (s)')
    axes[1,0].grid(True, alpha=0.3)
    axes[1,0].set_ylim(bottom=0)
    
    # Plot temperature
    axes[1,1].plot(timestamps, temp_c, 'orange', linewidth=1.5, alpha=0.8)
    axes[1,1].set_title('Temperature', fontsize=12, fontweight='bold')
    axes[1,1].set_ylabel('Temperature (°C)')
    axes[1,1].set_xlabel('Time (s)')
    axes[1,1].grid(True, alpha=0.3)
    
    # Add some statistics
    for ax in axes.flat:
        ax.tick_params(labelsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics (using absolute values)
    print(f"📈 Plotted {len(adc_df)} ADC measurements")
    print(f"📊 VBUS: {min(vbus_v):.3f}V - {max(vbus_v):.3f}V (avg: {np.mean(vbus_v):.3f}V)")
    print(f"📊 IBUS: {min(ibus_a):.3f}A - {max(ibus_a):.3f}A (avg: {np.mean(ibus_a):.3f}A)")
    print(f"📊 Power: {min(power_w):.3f}W - {max(power_w):.3f}W (avg: {np.mean(power_w):.3f}W)")
    print(f"📊 Temp: {min(temp_c):.1f}°C - {max(temp_c):.1f}°C (avg: {np.mean(temp_c):.1f}°C)")

✅ Project root found at: /home/okhsunrog/code/km003c-protocol-research
✅ Added to sys.path: /home/okhsunrog/code/km003c-protocol-research/analysis/scripts
✅ Successfully imported helpers.py
✅ Loaded 12,008 USB packets from /home/okhsunrog/code/km003c-protocol-research/usb_master_dataset.parquet


In [ ]:
if df is not None and not df.is_empty():
    # Get unique source files from the 'source_file' column
    source_files = ['All'] + sorted(df['source_file'].unique().to_list())

    # --- Create Widgets ---
    title = HTML("<h2>USB Transaction Analysis</h2>")
    
    source_dropdown = Dropdown(
        options=source_files,
        value='All',
        description='Source File:',
        style={'description_width': 'initial'},
        layout={'width': 'max-content'}
    )
    
    enum_checkbox = widgets.Checkbox(
        value=True,
        description='Hide Enumeration',
        disabled=False,
        indent=False
    )
    
    show_plots_checkbox = widgets.Checkbox(
        value=True,
        description='Show ADC Plots',
        disabled=False,
        indent=False
    )
    
    output_area = Output(layout={'height': '600px', 'overflow': 'scroll'})

    # --- Define Handler ---
    def on_change(change):
        """Callback to update analysis when a control value changes."""
        with output_area:
            output_area.clear_output(wait=True)
            
            selected_file = source_dropdown.value
            hide_enum = enum_checkbox.value
            show_plots = show_plots_checkbox.value
            
            if selected_file == 'All':
                filtered_df = df
                print(f"📊 Showing transactions for ALL source files.")
            else:
                filtered_df = df.filter(pl.col('source_file') == selected_file)
                print(f"📊 Showing transactions for: {selected_file}")

            # Display stats for the selected data
            if not filtered_df.is_empty():
                # Process and display transactions
                transactions = helpers.get_transactions(filtered_df, filter_out_enumeration=hide_enum)
                helpers.print_transaction_log(transactions, limit=200) # Limit to 200 for notebook display
                
                # Show ADC plots if requested
                if show_plots:
                    print("\n" + "="*80)
                    print("📈 ADC DATA VISUALIZATION")
                    print("="*80)
                    plot_adc_data(filtered_df, selected_file)
            else:
                print("No data available for this selection.")

    # --- Wire up and Display ---
    source_dropdown.observe(on_change, names='value')
    enum_checkbox.observe(on_change, names='value')
    show_plots_checkbox.observe(on_change, names='value')

    # Display widgets
    controls = widgets.HBox([source_dropdown, enum_checkbox, show_plots_checkbox])
    display(VBox([title, controls, output_area]))

    # Initial display - call on_change once with the initial state
    with output_area:
        selected_file = source_dropdown.value
        hide_enum = enum_checkbox.value
        show_plots = show_plots_checkbox.value
        
        if selected_file == 'All':
            filtered_df = df
            print(f"📊 Showing transactions for ALL source files.")
        else:
            filtered_df = df.filter(pl.col('source_file') == selected_file)
            print(f"📊 Showing transactions for: {selected_file}")

        if not filtered_df.is_empty():
            transactions = helpers.get_transactions(filtered_df, filter_out_enumeration=hide_enum)
            helpers.print_transaction_log(transactions, limit=200)
            
            if show_plots:
                print("\n" + "="*80)
                print("📈 ADC DATA VISUALIZATION")
                print("="*80)
                plot_adc_data(filtered_df, selected_file)
        else:
            print("No data available for this selection.")

elif df is not None and df.is_empty():
    print("Dataset is loaded but empty. No analysis to display.")
else:
    print("Dataset could not be loaded. Cannot build UI.")